# 자동 글쓰기평가 Automatic Writing Evaluation (AWE) 시스템을 만들자!  
1. AWE_1000_score.csv 파일을 읽고
2. content 칼럼의 글쓰기를 batch_size 4개씩 가져오는 Dataloader를 구현하시오

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('./drive/My Drive/NLP/04. Automatic Writting Evaluation on Linear Regression (1)/codes/')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import sklearn
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer


df = pd.read_csv("AWE_1000_score.csv")

corpus = df["content"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
df["content_vector"]= X.todense().tolist()


In [ ]:
class MyLinearModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = torch.nn.Linear(in_features=8093, out_features= 1)

  def forward(self, x):
    return self.linear(x)

myLin = MyLinearModel()

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.texts = dataframe['content_vector'].tolist()  # TF-IDF 벡터
        self.labels = dataframe['score'].tolist()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = torch.FloatTensor(self.texts[idx])
        label = torch.FloatTensor([self.labels[idx]])
        return text, label

dataset = TextDataset(df)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
optimizer = torch.optim.SGD(myLin.parameters(), lr=0.01)

nb_epochs = 10
for epoch in range(nb_epochs + 1):
    total_loss = 0
    for x_batch, y_batch in dataloader:
        hypothesis = myLin(x_batch)  # 모델을 통해 예측값 계산
        cost = F.mse_loss(hypothesis, y_batch)
        total_loss += cost.item()

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    print(epoch, total_loss / len(dataloader))


0 880.5347914042472
1 350.68982942199705
2 343.1767879104614
3 336.17813932037353
4 328.3824036331177
5 322.8665206985474
6 316.233527923584
7 310.47986128997803
8 304.7152969245911
9 298.73621852874754
10 293.2091926498413
